In [ ]:
# Import des libraries

from bigml.api import BigML
import pandas
from pandas import read_csv
import matplotlib

In [ ]:
# Modification du fichier Train de Kaggle

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('train_full.csv')

In [ ]:
# Modification du fichier Test de Kaggle

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('test_full.csv')

In [ ]:
# Connexion au projet via l'API de BIG ML

api = BigML('MAXIMEFAVOT', '69757fb108f0bd7a8678e0b66e3de66ae6478c81', project='project/5d9e081659f5c320e1000597')

In [ ]:
# Creation des sources sur BigML

source_train = api.create_source('train_full.csv')
trainfull_dataset = api.create_dataset(source_train)

source_test = api.create_source('test_full.csv')
testfull_dataset = api.create_dataset(source_test)

In [ ]:
# Création du split

train_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Training_split", "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Test_split", "sample_rate": 0.8, "seed": "my seed", "out_of_bag": True})

In [ ]:
# Création du modele Ensemble

ensemble = api.create_ensemble(train_dataset, {"name": "ensemble", "objective_field": "SeriousDlqin2yrs"})

In [ ]:
# Batch Prediction et Download

batch_prediction = api.create_batch_prediction(ensemble, testfull_dataset,{"all_fields": True, "probabilities": True})

In [ ]:
# Récuperation des evaluations sur l'API

evaluation = api.create_evaluation(ensemble, test_dataset)
api.ok(evaluation)
api.pprint(evaluation['object']['result']['model']['accuracy'])
api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])
api.pprint(evaluation['object']['result']['model']['confusion_matrix'])

In [ ]:
api.download_batch_prediction(batch_prediction, filename='Prediction_kaggle.csv')

In [ ]:
# Mise en forme pour soumission Kaggle

df = read_csv('Prediction_kaggle.csv', index_col=0)
df2 = df[['1 probability']]
df2.index.names = ['Id'] 
df2.rename(columns={"1 probability": "Probability"}, inplace= True)
df2.to_csv('Kaggle-submission.csv')

In [ ]:
# Soumission Kaggle

import kaggle
submission_file = "Kaggle-submission.csv"
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMesomeCredit")

